In [1]:
import nltk
nltk.download("all")

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     C:\Users\Philippa\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\abc.zip.
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     C:\Users\Philippa\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     C:\Users\Philippa\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping taggers\averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     C:\Users\Philippa\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers\averaged_perceptron_tagger_eng.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     C:\Users\Philippa\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]  

True

In [47]:
import pandas as pd
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

df = pd.read_csv(r"C:\Users\Philippa\Documents\GitHub\webscraper\Scraper\NL_Parsed_Reviews.csv")

df_NL = df.copy()
df_NL

,name,date,rating,review
0,Ughduty,"Jun 19, 2025",7,Never played the game. Metacritic assigned my ...
1,tarquinthegreat,"Apr 28, 2025",10,absolutely banging ‎xxxxxxxxxxxxxxxxxxxxxxxxxx...
2,Gamewing,"Apr 21, 2025",6,"First couple hours are fun, getting your first..."
3,Z4ry,"Mar 24, 2025",9,100+ hours\n- Great evolution from previous ti...
4,Lucaq,"Mar 20, 2025",10,Perfect in every way.\nAaaaaaaaaaaaaaaaaaaaaaa...
...,...,...,...,...
209,Autosaver,"Jun 9, 2013",10,Animal Crossing has gone a long way since the ...
210,KirkyDaTurkey,"Jun 9, 2013",9,"Based on Day 1, this game is definitely a 9 so..."
211,EFYevan,"Jun 9, 2013",10,Much different than the previous DS version I...
212,L0stR3negade,"Jun 9, 2013",9,Refreshing style of game to play. Its hard to ...


## Basic Sentiment Analysis

In [48]:
# remove date and name as they aren't relevant for sentiment analysis
df_NL = df_NL.drop(columns=["name","date"])
df_NL

,rating,review
0,7,Never played the game. Metacritic assigned my ...
1,10,absolutely banging ‎xxxxxxxxxxxxxxxxxxxxxxxxxx...
2,6,"First couple hours are fun, getting your first..."
3,9,100+ hours\n- Great evolution from previous ti...
4,10,Perfect in every way.\nAaaaaaaaaaaaaaaaaaaaaaa...
...,...,...
209,10,Animal Crossing has gone a long way since the ...
210,9,"Based on Day 1, this game is definitely a 9 so..."
211,10,Much different than the previous DS version I...
212,9,Refreshing style of game to play. Its hard to ...


In [49]:
df_NL.isnull().sum()

rating    0
review    0
dtype: int64

In [50]:
df_NL["rating"].value_counts()

rating
10    99
9     67
8     23
7      7
6      5
5      5
4      3
0      2
1      2
2      1
Name: count, dtype: int64

In [51]:
sia = SentimentIntensityAnalyzer()
df_NL["scores"] = df["review"].apply(lambda review:sia.polarity_scores(review))

df_NL.head()

,rating,review,scores
0,7,Never played the game. Metacritic assigned my ...,"{'neg': 0.097, 'neu': 0.903, 'pos': 0.0, 'comp..."
1,10,absolutely banging ‎xxxxxxxxxxxxxxxxxxxxxxxxxx...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
2,6,"First couple hours are fun, getting your first...","{'neg': 0.101, 'neu': 0.734, 'pos': 0.166, 'co..."
3,9,100+ hours\n- Great evolution from previous ti...,"{'neg': 0.0, 'neu': 0.526, 'pos': 0.474, 'comp..."
4,10,Perfect in every way.\nAaaaaaaaaaaaaaaaaaaaaaa...,"{'neg': 0.0, 'neu': 0.519, 'pos': 0.481, 'comp..."


In [52]:
# compound the value to compare
df_NL["compound"] = df_NL["scores"].apply(lambda score_dict: score_dict["compound"])
df_NL.head()

,rating,review,scores,compound
0,7,Never played the game. Metacritic assigned my ...,"{'neg': 0.097, 'neu': 0.903, 'pos': 0.0, 'comp...",-0.2584
1,10,absolutely banging ‎xxxxxxxxxxxxxxxxxxxxxxxxxx...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000
2,6,"First couple hours are fun, getting your first...","{'neg': 0.101, 'neu': 0.734, 'pos': 0.166, 'co...",0.7303
3,9,100+ hours\n- Great evolution from previous ti...,"{'neg': 0.0, 'neu': 0.526, 'pos': 0.474, 'comp...",0.8720
4,10,Perfect in every way.\nAaaaaaaaaaaaaaaaaaaaaaa...,"{'neg': 0.0, 'neu': 0.519, 'pos': 0.481, 'comp...",0.5719


In [53]:
# extrapolate the overall sentiment (if compound > 0 then positive, otherwise neg)
df_NL["sentiment"] = df_NL["compound"].apply(lambda c:"pos" if c>=0 else "neg")
df_NL.head()

,rating,review,scores,compound,sentiment
0,7,Never played the game. Metacritic assigned my ...,"{'neg': 0.097, 'neu': 0.903, 'pos': 0.0, 'comp...",-0.2584,neg
1,10,absolutely banging ‎xxxxxxxxxxxxxxxxxxxxxxxxxx...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,pos
2,6,"First couple hours are fun, getting your first...","{'neg': 0.101, 'neu': 0.734, 'pos': 0.166, 'co...",0.7303,pos
3,9,100+ hours\n- Great evolution from previous ti...,"{'neg': 0.0, 'neu': 0.526, 'pos': 0.474, 'comp...",0.8720,pos
4,10,Perfect in every way.\nAaaaaaaaaaaaaaaaaaaaaaa...,"{'neg': 0.0, 'neu': 0.519, 'pos': 0.481, 'comp...",0.5719,pos


In [54]:
# convert ratings to pos or neg for accuracy metrics
df_NL["rating"] = df_NL["rating"].replace(to_replace=(0, 1, 2, 3, 4, 5), value="neg")
df_NL["rating"] = df_NL["rating"].replace(to_replace=(6, 7, 8, 9, 10), value="pos")

df_NL

,rating,review,scores,compound,sentiment
0,pos,Never played the game. Metacritic assigned my ...,"{'neg': 0.097, 'neu': 0.903, 'pos': 0.0, 'comp...",-0.2584,neg
1,pos,absolutely banging ‎xxxxxxxxxxxxxxxxxxxxxxxxxx...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,pos
2,pos,"First couple hours are fun, getting your first...","{'neg': 0.101, 'neu': 0.734, 'pos': 0.166, 'co...",0.7303,pos
3,pos,100+ hours\n- Great evolution from previous ti...,"{'neg': 0.0, 'neu': 0.526, 'pos': 0.474, 'comp...",0.8720,pos
4,pos,Perfect in every way.\nAaaaaaaaaaaaaaaaaaaaaaa...,"{'neg': 0.0, 'neu': 0.519, 'pos': 0.481, 'comp...",0.5719,pos
...,...,...,...,...,...
209,pos,Animal Crossing has gone a long way since the ...,"{'neg': 0.079, 'neu': 0.76, 'pos': 0.161, 'com...",0.6908,pos
210,pos,"Based on Day 1, this game is definitely a 9 so...","{'neg': 0.04, 'neu': 0.843, 'pos': 0.117, 'com...",0.9141,pos
211,pos,Much different than the previous DS version I...,"{'neg': 0.034, 'neu': 0.848, 'pos': 0.118, 'co...",0.7850,pos
212,pos,Refreshing style of game to play. Its hard to ...,"{'neg': 0.134, 'neu': 0.613, 'pos': 0.253, 'co...",0.5789,pos


In [55]:
from sklearn.metrics import accuracy_score

accuracy_score(df_NL["rating"], df_NL["sentiment"])

0.9158878504672897

As a very simple sentiment analysis (with no changes to the model or pre-processing of text), the accuracy score is surprisingly good. But It would be best to experiment with more sophisticated models, such as BERT.

I also need to note that the rating system is a bit trickier to work with. Sometimes, the sentiment of text does not necessarily match the rating score a user has provided. As the rating is also out of 10, it would be better to include a "neutral" sentiment instead of only positive and negative. 